In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Dropout,Activation
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.utils import to_categorical
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from tensorflow.keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from tensorflow.keras.preprocessing import sequence, text
from tensorflow.keras.callbacks import EarlyStopping


import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff

In [2]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [3]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test_x.csv')

In [4]:
train

,index,text,author
0,0,"He was almost choking. There was so much, so m...",3
1,1,"“Your sister asked for it, I suppose?”",2
2,2,"She was engaged one day as she walked, in per...",1
3,3,"The captain was in the porch, keeping himself ...",4
4,4,"“Have mercy, gentlemen!” odin flung up his han...",3
...,...,...,...
54874,54874,"“Is that you, Mr. Smith?” odin whispered. “I h...",2
54875,54875,"I told my plan to the captain, and between us ...",4
54876,54876,"""Your sincere well-wisher, friend, and sister...",1
54877,54877,“Then you wanted me to lend you money?”,3


In [5]:
def roc_auc(predictions,target):
    '''
    This methods returns the AUC Score when given the Predictions
    and Labels
    '''
    
    fpr, tpr, thresholds = metrics.roc_curve(target, predictions)
    roc_auc = metrics.auc(fpr, tpr)
    return roc_auc

In [6]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.text.values, train.author.values, 
                                                  stratify=train.author.values, 
                                                  random_state=42, 
                                                  test_size=0.2, shuffle=True)

In [7]:
#토큰화
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [8]:
# using keras tokenizer here
token = Tokenizer(num_words=None)
max_len = 1500

token.fit_on_texts(list(xtrain) + list(xvalid))

xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)

#zero pad the sequences
xtrain_pad = pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index

In [9]:
train['text'] = train['text'].str.lower()
test['text'] = test['text'].str.lower()
X_train = train['text'].values
X_test = test['text'].values
y = train['author'].values

tokenizer = Tokenizer(num_words = None)
max_len = 1500

tokenizer.fit_on_texts(X_train)

train_sequences = tokenizer.texts_to_sequences(X_train)
test_sequences = tokenizer.texts_to_sequences(X_test)

#zero pad the sequences
trn = pad_sequences(train_sequences,maxlen=max_len)
tst = pad_sequences(test_sequences, maxlen=max_len)

word_index = tokenizer.word_index

In [10]:
#Word Embeddings

In [11]:
# load the GloVe vectors in a dictionary:

embeddings_index = {}
f = open('./data/glove.6B.300d.txt','r',encoding='utf-8')
for line in tqdm(f):
    values = line.split(' ')
    word = values[0]
    coefs = np.asarray([float(val) for val in values[1:]])
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

400000it [00:37, 10608.75it/s]

Found 400000 word vectors.


In [12]:
# create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|██████████| 42330/42330 [00:00<00:00, 205371.02it/s]


In [13]:
%%time
with strategy.scope():
    
    # A simple LSTM with glove embeddings and one dense layer
    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))

    model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))
    model.add(Dense(5, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
    
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1500, 300)         12699300  
_________________________________________________________________
lstm (LSTM)                  (None, 100)               160400    
_________________________________________________________________
dense (Dense)                (None, 5)                 505       
Total params: 12,860,205
Trainable params: 160,905
Non-trainable params: 12,699,300
_________________________________________________________________
CPU times: user 428 ms, sys: 380 ms, total: 808 ms
Wall time: 664 ms


In [14]:
from tensorflow.keras.utils import to_categorical
ytrain= to_categorical (ytrain)
yvalid= to_categorical (yvalid)

In [15]:
ytrain

array([[0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       ...,
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.]], dtype=float32)

In [16]:
model.fit(xtrain_pad, ytrain, epochs=5, batch_size=64*strategy.num_replicas_in_sync)

Epoch 1/5
686/686 [==============================] - 1882s 3s/step - loss: 0.4281 - accuracy: 0.4504
Epoch 2/5
686/686 [==============================] - 1885s 3s/step - loss: 0.3668 - accuracy: 0.5563
Epoch 3/5
686/686 [==============================] - 1882s 3s/step - loss: 0.3365 - accuracy: 0.6006
Epoch 4/5
686/686 [==============================] - 1882s 3s/step - loss: 0.3133 - accuracy: 0.6362
Epoch 5/5
686/686 [==============================] - 1874s 3s/step - loss: 0.2965 - accuracy: 0.6606


In [ ]:
scores = model.predict(xvalid_pad)
print("Auc: %.4f%%" % (roc_auc(scores,yvalid)))

In [17]:
p_tst = model.predict(tst)

In [19]:
sub = pd.read_csv('./data/sample_submission.csv', index_col=0)
sub[sub.columns] = p_tst
sub.to_csv('ZerotoTransLstm1.csv')#0.7768

In [16]:
to_categorical(y)

array([[0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       ...,
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.]], dtype=float32)

In [17]:
model.fit(trn, to_categorical(y), epochs=5, batch_size = 64*strategy.num_replicas_in_sync)

Epoch 1/5
858/858 [==============================] - 2393s 3s/step - loss: 0.3854 - accuracy: 0.5293
Epoch 2/5
858/858 [==============================] - 2372s 3s/step - loss: 0.3156 - accuracy: 0.6373
Epoch 3/5
858/858 [==============================] - 2378s 3s/step - loss: 0.2831 - accuracy: 0.6818
Epoch 4/5
858/858 [==============================] - 2361s 3s/step - loss: 0.2618 - accuracy: 0.7084
Epoch 5/5
858/858 [==============================] - 2363s 3s/step - loss: 0.2461 - accuracy: 0.7303


In [18]:
p_tst = model.predict(tst)

In [22]:
sub = pd.read_csv('./data/sample_submission.csv', index_col=0)
sub[sub.columns] = p_tst
sub.to_csv('ZerotoTransLstm2.csv')#0.563